In [2]:
#!/usr/bin/python3
import math

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    ControllabilityMatrix,
    DiagramBuilder,
    Linearize,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    Parser,
    Saturation,
    SceneGraph,
    Simulator,
    StartMeshcat,
    WrapToSystem,
    PointCloud,
    MeshcatPointCloudVisualizer,
    BaseField,
    Fields,
    Rgba
)
from pydrake.examples import (
    AcrobotGeometry,
    AcrobotInput,
    AcrobotPlant,
    AcrobotState,
    QuadrotorGeometry,
    QuadrotorPlant,
    StabilizingLQRController,
)
from pydrake.symbolic import Variable
from pydrake.systems.primitives import SymbolicVectorSystem
from pydrake.solvers import MathematicalProgram, Solve

from quad_traj_opt.trajectory_optimization import TrajectoryOptimization
from quad_traj_opt.differential_flatness import trajectory_to_state
import quad_traj_opt.trajectory_controller
from quad_traj_opt.utils import waypoints_to_point_cloud

import quadrotor_params as p
import waypoint_library


In [4]:
# waypoints = waypoint_library.waypoints_straight
waypoints = waypoint_library.waypoints_flip/3
headings = np.zeros((1, waypoints.shape[1]))

traj_gen = TrajectoryOptimization(waypoints, headings, ts=0.5)
traj_gen.solve()

# t = np.linspace(0, traj_gen.tf, 1000)
dt = .01
t = np.arange(0.0, traj_gen.tf, dt)

In [5]:
running_as_notebook = True
meshcat = StartMeshcat()
builder = DiagramBuilder()
waypoint_vis = waypoints_to_point_cloud(waypoints, 
    pts_per_edge=p.keyframes_pts_per_edge, half_width=p.keyframes_half_width)

plant = builder.AddSystem(QuadrotorPlant(
    m_arg=p.m_kg,
    L_arg=p.L_m,
    I_arg=p.J,
    kF_arg=p.kF,
    kM_arg=p.kM
))

point_cloud = PointCloud(new_size=len(t), fields=Fields(BaseField.kXYZs))
thing = point_cloud.mutable_xyzs()
for i, ti in enumerate(t):
    thing[:,i] = traj_gen.eval_trajectory(ti, 0)
meshcat.SetObject("/point_cloud/", point_cloud, point_size=.05, rgba=Rgba(1.0, 0.0, 0.5, 1.0))
meshcat.SetObject("/waypoints/", waypoint_vis, point_size=.05, rgba=Rgba(0.0, 0.5, 0.5, 1.0))

# Set up visualization in MeshCat
scene_graph = builder.AddSystem(SceneGraph())
QuadrotorGeometry.AddToBuilder(
    builder, plant.get_output_port(0), scene_graph
)
meshcat.Delete()
meshcat.ResetRenderMode()
meshcat.SetProperty("/Background", "visible", False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
# end setup for visualization

diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
context = simulator.get_mutable_context()
# print(context.GetInputPort())

INFO:drake:Meshcat listening for connections at http://localhost:7004


In [6]:
# Simulate
for i in range(3):
    context.SetTime(0.0)
    simulator.Initialize()
    context.SetContinuousState(np.zeros(12))

    
    
    for ti in t[0:]:
        try:
            simulator.AdvanceTo(ti)
        except:
            simulator.Initialize()
        traj_matrix = traj_gen.eval_full_trajectory(ti)
        x = trajectory_to_state(traj_matrix, plant)
        # x_part = np.vstack([x[:6,:], np.zeros((3,1)), x[9:,:]])
        # x_part = np.vstack([x[:3,:], x[3:6,:], x[6:9, :], x[9:,:]])
        # x_fixed = np.vstack([x[:2,:], [-x[2,:]], x[3:8], [-x[8,:]], x[9:,:]])
        context.SetContinuousState(x)